In [ ]:
import glob, os
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

In [ ]:
print(os.getcwd())

In [ ]:
os.chdir('../data/FDA_Validation_Set_Results')

In [ ]:
directories = ['./Predictions_ECFP4','./Predictions_MD','.Predictions_PT']

In [ ]:
def metrics_func(TP,FP,FN,TN):

    
    if TN == 0:
        specificity=0
        recall = TP/(TP+FN)
        precision = TP/(TP+FP)
        if (TN+FN) == 0:
            MCC = 0
        else:
            MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))

        
    elif TP == 0:
        precision = 0
        recall=0
        specificity = TN/(TN+FP)
        if (FP+TP) == 0:
            MCC = 0
        else:
            MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))


        
#    elif (TN+FN) ==0 or (FP+TP) == 0:
#        MCC=0
        
    else:
        specificity = TN/(TN+FP)
        recall = TP/(TP+FN)
        MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))
        precision = TP/(TP+FP)
    
    
    balanced_accuracy = (specificity+recall)/2
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    
    return precision,recall,specificity,balanced_accuracy,accuracy,MCC

In [ ]:
faulty_file_dirs = []
for result_dir in directories:
    #df = pd.DataFrame(data=None, columns=['model','precision','recall','balanced_accuracy','MCC'])
    list_of_lists = []
    for file in os.listdir(result_dir):
        if file.endswith(".txt"):
            print(os.path.join(result_dir, file))
            f = open(os.path.join(result_dir, file), 'r')
            try:

                content = [line.rstrip('\n') for line in f]
                print(content)
                
                
                size=sum(1 for _ in content)
                print(size)
                
                
                print(type(content[1].split(",")[0]),content[1].split(",")[0])
                
                p = content[1].split(",")[0].replace('"', '')
                print(p)
                if p == str("1"): # if never predicted 0 missing line
                    print("wow1")
                    TN = 0
                    FN = 0
                    FP = int(content[1].split(",")[1])
                    TP = int(content[1].split(",")[2])
                    
                elif p == str("0") and size == 2 : # if never predicted 1
                    print("wow2")
                    TN = int(content[1].split(",")[1])
                    FN = int(content[1].split(",")[2])
                    FP = 0
                    TP = 0
                    print(TN,FN,FP,TP)

                else:

                    TN = int(content[1].split(",")[1])
                    FN = int(content[1].split(",")[2])
                    FP = int(content[2].split(",")[1])
                    TP = int(content[2].split(",")[2])


                p,r,sen,ba,a,mcc = metrics_func(TP,FP,FN,TN)

                list_of_lists.append([file,p,r,sen,ba,a,mcc])
                
            except:
                print('ERROR')
                break
                
                faulty_file_dirs.append(os.path.join(result_dir, file))
                list_of_lists.append([file,None,None,None,None,None,None])
                
            finally:
                f.close()
    df = pd.DataFrame(list_of_lists,columns=['model','precision','recall','specificity','balanced_accuracy','accuracy','MCC'])
    df['model'] = df['model'].str[4:]
    df['model'] = df['model'].str[:-4]
    df.replace('_', '.', regex=True, inplace=True)
    #df['model'] = df['model'].replace("_", ".")
    df.rename(columns={"model": "splits"},inplace=True)
    df.to_csv(result_dir+"/ambiguous_results.csv")

In [ ]:
faulty_file_dirs

In [ ]:
df